In [ ]:
import numpy as np
from tensorflow import keras as ks
import pandas as pd

---

In [ ]:
from scipy.sparse import load_npz

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
X_train = load_npz('drive/MyDrive/Colab Notebooks/X_train.npz')
X_train

<63768x63822 sparse matrix of type '<class 'numpy.float64'>'
	with 1992232 stored elements in Compressed Sparse Row format>

In [ ]:
X_test = load_npz('drive/MyDrive/Colab Notebooks/X_test.npz')
X_test

<21257x63822 sparse matrix of type '<class 'numpy.float64'>'
	with 651895 stored elements in Compressed Sparse Row format>

In [ ]:
y_train = np.load('drive/MyDrive/Colab Notebooks/y_train.npy')
y_train.shape

(63768,)

In [ ]:
y_test = np.load('drive/MyDrive/Colab Notebooks/y_test.npy')
y_test.shape

(21257,)

---

## Keras

In [ ]:
from keras import backend as K
def r_square(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [ ]:
from keras.layers import Dense, Input
from keras.models import Model

model_in = Input(shape=(63822), sparse=True)
out = Dense(512, activation='relu')(model_in)
out = Dense(192, activation='relu')(out)
out = Dense(64, activation='relu')(out)
out = Dense(1)(out)
model = Model(model_in, out)

model.compile(loss='mean_squared_error', optimizer=ks.optimizers.Adam(lr=3e-3), metrics=[r_square])
model.summary()

Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 63822)]           0         
_________________________________________________________________
dense_48 (Dense)             (None, 512)               32677376  
_________________________________________________________________
dense_49 (Dense)             (None, 192)               98496     
_________________________________________________________________
dense_50 (Dense)             (None, 64)                12352     
_________________________________________________________________
dense_51 (Dense)             (None, 1)                 65        
Total params: 32,788,289
Trainable params: 32,788,289
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from keras.callbacks import EarlyStopping

In [ ]:
es = EarlyStopping(monitor='val_r_square', mode='max', patience=2, restore_best_weights=True)

In [ ]:
model.fit(x=X_train, y=y_train, batch_size=128, validation_data=(X_test, y_test), callbacks=[es], epochs=1_000)

Epoch 1/1000
499/499 [==============================] - 31s 60ms/step - loss: 266.6169 - r_square: -0.8187 - val_loss: 45.4460 - val_r_square: 0.6762
Epoch 2/1000
499/499 [==============================] - 31s 62ms/step - loss: 27.1971 - r_square: 0.8057 - val_loss: 41.8835 - val_r_square: 0.7014
Epoch 3/1000
499/499 [==============================] - 29s 58ms/step - loss: 15.6681 - r_square: 0.8889 - val_loss: 39.3233 - val_r_square: 0.7194
Epoch 4/1000
499/499 [==============================] - 31s 62ms/step - loss: 11.1338 - r_square: 0.9202 - val_loss: 38.1920 - val_r_square: 0.7269
Epoch 5/1000
499/499 [==============================] - 29s 58ms/step - loss: 8.5906 - r_square: 0.9377 - val_loss: 37.9481 - val_r_square: 0.7286
Epoch 6/1000
499/499 [==============================] - 31s 63ms/step - loss: 7.0054 - r_square: 0.9498 - val_loss: 37.5996 - val_r_square: 0.7315
Epoch 7/1000
499/499 [==============================] - 30s 60ms/step - loss: 5.7658 - r_square: 0.9586 - val_lo

In [ ]:
model.fit(x=X_train, y=y_train, batch_size=256, validation_data=(X_test, y_test), callbacks=[es], epochs=1_000)

Epoch 1/1000
250/250 [==============================] - 16s 65ms/step - loss: 13.4508 - r_square: 0.9049 - val_loss: 36.2998 - val_r_square: 0.7437
Epoch 2/1000
250/250 [==============================] - 18s 70ms/step - loss: 4.7117 - r_square: 0.9666 - val_loss: 35.1279 - val_r_square: 0.7516
Epoch 3/1000
250/250 [==============================] - 16s 63ms/step - loss: 2.3563 - r_square: 0.9833 - val_loss: 35.3096 - val_r_square: 0.7503
Epoch 4/1000
250/250 [==============================] - 16s 63ms/step - loss: 1.5573 - r_square: 0.9889 - val_loss: 36.5672 - val_r_square: 0.7417


In [ ]:
model.fit(x=X_train, y=y_train, batch_size=512, validation_data=(X_test, y_test), callbacks=[es], epochs=1_000)

Epoch 1/1000
125/125 [==============================] - 9s 73ms/step - loss: 2.3428 - r_square: 0.9835 - val_loss: 35.0018 - val_r_square: 0.7531
Epoch 2/1000
125/125 [==============================] - 9s 75ms/step - loss: 1.3013 - r_square: 0.9908 - val_loss: 35.1682 - val_r_square: 0.7520
Epoch 3/1000
125/125 [==============================] - 9s 75ms/step - loss: 0.8671 - r_square: 0.9939 - val_loss: 35.3560 - val_r_square: 0.7507


In [ ]:
model.save('drive/MyDrive/Colab Notebooks/my_model.h5')

---

In [ ]:
X_eval = load_npz('drive/MyDrive/Colab Notebooks/X_eval.npz')
X_eval

<30186x63822 sparse matrix of type '<class 'numpy.float64'>'
	with 932703 stored elements in Compressed Sparse Row format>

In [ ]:
y_eval = model.predict(X_eval)

In [ ]:
pd.DataFrame(y_eval, columns=['Predicted']).to_csv('output.csv', index_label='Id')